In [2]:
import sagemaker
import boto3
from sagemaker.session import s3_input, Session


sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('./data-clarify/final_data.csv')


In [5]:
training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)


In [6]:
training_data

,user_id,order_number,days_since_prior_order,product_id,reordered,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,...,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
986006,147050,35,7.0,41220,1,9.031196,1154,750,0.649913,1154,...,0,0,0,0,0,0,0,0,0,0
744924,110726,17,30.0,36107,1,6.035433,254,176,0.692913,254,...,0,0,0,0,0,0,0,0,0,0
994107,148215,7,30.0,7192,1,9.666667,9,5,0.555556,9,...,0,0,0,0,0,0,0,0,0,0
1235245,183809,7,22.0,40604,1,10.193237,1449,858,0.592133,1449,...,0,0,0,0,0,0,0,1,0,0
686997,102164,4,30.0,18027,0,8.779370,698,442,0.633238,698,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130365,19415,21,12.0,20211,1,8.000000,15,5,0.333333,15,...,0,0,0,0,0,0,0,0,1,0
616591,91678,11,5.0,44233,1,10.236364,110,62,0.563636,110,...,0,0,0,0,0,0,1,0,0,0
1055194,157404,4,24.0,20518,1,3.575221,113,85,0.752212,113,...,0,0,0,0,0,0,0,0,0,0
1139006,169667,12,21.0,36183,0,7.369565,46,19,0.413043,46,...,0,0,0,0,0,0,0,0,0,0


# Target variable must be in 1st column and data should be in csv format for XG boost

In [7]:
col_name="reordered"
first_col = training_data.pop(col_name)

In [8]:
training_data.insert(0, col_name, first_col)

# Header must be removed

In [9]:
training_data.columns = range(training_data.shape[1])
print (training_data)

         0       1   2     3      4          5     6    7         8     9   \
986006    1  147050  35   7.0  41220   9.031196  1154  750  0.649913  1154   
744924    1  110726  17  30.0  36107   6.035433   254  176  0.692913   254   
994107    1  148215   7  30.0   7192   9.666667     9    5  0.555556     9   
1235245   1  183809   7  22.0  40604  10.193237  1449  858  0.592133  1449   
686997    0  102164   4  30.0  18027   8.779370   698  442  0.633238   698   
...      ..     ...  ..   ...    ...        ...   ...  ...       ...   ...   
130365    1   19415  21  12.0  20211   8.000000    15    5  0.333333    15   
616591    1   91678  11   5.0  44233  10.236364   110   62  0.563636   110   
1055194   1  157404   4  24.0  20518   3.575221   113   85  0.752212   113   
1139006   0  169667  12  21.0  36183   7.369565    46   19  0.413043    46   
29828     0    4390  26  30.0  12276   9.224880   418  198  0.473684   418   

         ...  51  52  53  54  55  56  57 58  59  60  
986006   

In [10]:
training_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
986006,1,147050,35,7.0,41220,9.031196,1154,750,0.649913,1154,...,0,0,0,0,0,0,0,0,0,0
744924,1,110726,17,30.0,36107,6.035433,254,176,0.692913,254,...,0,0,0,0,0,0,0,0,0,0
994107,1,148215,7,30.0,7192,9.666667,9,5,0.555556,9,...,0,0,0,0,0,0,0,0,0,0
1235245,1,183809,7,22.0,40604,10.193237,1449,858,0.592133,1449,...,0,0,0,0,0,0,0,1,0,0
686997,0,102164,4,30.0,18027,8.779370,698,442,0.633238,698,...,0,0,0,0,0,0,0,0,0,0


In [11]:
path_train = "./data-clarify/train_data.csv"
training_data.to_csv(path_train, index=False, header=True)

In [12]:
import time

timestamp = int(time.time())

train_data_s3_uri = sess.upload_data(bucket=bucket, key_prefix="traindata-{}".format(timestamp), path=path_train)
train_data_s3_uri

's3://sagemaker-us-east-1-222711858116/traindata-1650247194/train_data.csv'

In [13]:
path_test = "./data-clarify/test_data.csv"
testing_data.to_csv(path_test, index=False, header=True)

In [14]:
import time

timestamp = int(time.time())

test_data_s3_uri = sess.upload_data(bucket=bucket, key_prefix="testdata-{}".format(timestamp), path=path_test)
test_data_s3_uri

's3://sagemaker-us-east-1-222711858116/testdata-1650247216/test_data.csv'

In [15]:
container = sagemaker.image_uris.retrieve(region= boto3.Session().region_name, framework= 'xgboost', version = 'latest')

In [16]:
prefix = 'xgboostframework'

xgb= sagemaker.estimator.Estimator(container, role, train_instance_count =1, train_instance_type = 'ml.m4.xlarge', output_path = 's3://{}/{}/output'.format(bucket,prefix), sagemaker_session =sess)

xgb.set_hyperparameters(max_depth= 5,
        eta=0.2,
        gamma=4,
        min_child_weight=6,
        subsample=0.8,
        objective= 'binary:logistic',
        num_round=100)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [17]:
content_type = "csv"
train_input = s3_input(train_data_s3_uri, content_type=content_type)
test_input = s3_input(test_data_s3_uri, content_type=content_type)





The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [18]:
model= xgb.fit({'train':train_input})

2022-04-18 02:00:34 Starting - Starting the training job...ProfilerReport-1650247234: InProgress
......
2022-04-18 02:02:00 Starting - Preparing the instances for training......
2022-04-18 02:03:01 Downloading - Downloading input data......
2022-04-18 02:04:01 Training - Downloading the training image.....Arguments: train
[2022-04-18:02:04:43:INFO] Running standalone xgboost training.
[2022-04-18:02:04:43:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-04-18:02:04:43:INFO] File size need to be processed in the node: 358.84mb. Available memory size in the node: 8473.24mb
[2022-04-18:02:04:43:INFO] Determined delimiter of CSV input is ','
[02:04:43] S3DistributionType set as FullyReplicated
[02:04:46] 1107694x60 matrix with 66461640 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,

2022-04-18 02:05:01 Training - Training image download completed. Training in progress.[02:04:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 ex